In [1]:
%%capture
%run RC4.ipynb
%run SDES.ipynb

import socket
from _thread import start_new_thread

key = None
cipher = None
ciphers = {'rc4': RC4, 'sdes': SDES}
server = None

def listen_server():
    while server:
        try:
            show_msg(server.recv(2048))
        except:
            pass

def stop_client():
    global key
    global cipher
    global server
    key = None
    cipher = None
    if server:
        server.close()
        server = None

def start_client(ip, port):
    global server
    stop_client()
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.connect((ip, port))
    start_new_thread(listen_server, ())

def command_check(text):
    global key
    global cipher
    if text[:6] == '\crypt':
        cipher = ciphers.get(text[7:text.find(' ', 8)])
        if cipher:
            key = text[text.find(' ', 8)+1:]
        else:
            key = None
        return False
    return True

def show_msg(text):
    text = str(text, 'utf-8')
    if command_check(text) and cipher:
        print('Server :', cipher(key, text))
    else:
        print('Server :', text)

def send_msg(text):
    try:
        if command_check(text) and cipher:
            server.send(cipher(key, text).encode())
        else:
            server.send(bytes(text, 'utf-8'))
    except:
        pass

In [2]:
start_client('localhost', 5354)

In [3]:
send_msg('Hello')

Server : Hello


In [4]:
stop_client()